In [28]:
import os
import argparse
import pandas as pd
import numpy as np
import pickle
import sys

In [29]:
sys.path.insert(0, './../utils/')
#from utils import *
#from utils_1 import *
from utils_ma import *

In [30]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [31]:
from keras.models import load_model
#from theano.ifelse import ifelse #added this

In [32]:
%pwd

'/home/hoang/Dropbox/process-sequence/experiment'

In [33]:
name = 'helpdesk'
#name = 'bpi_12_w'
args = {
    'inputdir': './../input/{}/'.format(name),   
    'outputdir': './output_files/{}/'.format(name),
    'modelname': 'model_349-1.72.h5',
}

args = argparse.Namespace(**args)

In [34]:
test = pd.read_csv(args.inputdir+'test.csv')
test = transformDf(test)

In [35]:
with open(args.inputdir+'parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [36]:
with open(args.inputdir+'preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [37]:
model = load_model(args.outputdir+args.modelname)

In [38]:
test_groupByCase = test.groupby(['CaseID'])

#get features all data
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

#get output all data
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

# Check longest case --> suffix: 2 to test_len-2
test_len = findLongestLength(test_groupByCase)

KeyError: 'SMA_16'

In [20]:
def getSuffix(suffix, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t):
    sentences_2 = []
    sentences_t_2 = []
    sentences_t2_2 = []
    sentences_t3_2 = []
    sentences_t4_2 = []
    next_chars_2 = []
    next_chars_t_2 = []

    for i in range(len(sentences)):
        if len(sentences[i]) == suffix and next_chars[i] != 'EOS':
            sentences_2.append(sentences[i])
            sentences_t_2.append(sentences_t[i])
            sentences_t2_2.append(sentences_t2[i])
            sentences_t3_2.append(sentences_t3[i])
            sentences_t4_2.append(sentences_t4[i])
            next_chars_2.append(next_chars[i])
            next_chars_t_2.append(next_chars_t[i])
    return sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2

In [21]:
def evalAct(true_label, pred_prob, targetchartoindice):
    #for log-loss: get probabilities
    gt_a = one_hot_encode(true_label, targetchartoindice)
    log_loss = metrics.log_loss(gt_a, pred_prob)
    print('Log-loss: {}'.format(log_loss))
    
    #for accuracy: get labels
    gt_a_label = true_label
    pred_a_label = getLabel(pred_prob, targetchartoindice)
    acc = metrics.accuracy_score(gt_a_label, pred_a_label)
    print('Accuracy: {}%'.format(acc*100))
    acc_3 = get_top3_accuracy(pred_prob, gt_a_label, targetchartoindice)
    print('Top 3 accuracy: {}%'.format(acc_3*100))

In [22]:
def evalTime(true_time, pred_time, divisor):
    #get time
    gt_t = true_time
    pred_t = inverseTime(pred_time, divisor)
    
    mse = metrics.mean_squared_error(gt_t, pred_t)
    mae = metrics.mean_absolute_error(gt_t, pred_t)
    median = metrics.median_absolute_error(gt_t, pred_t)
    
    print('Mean Squared Error: {0}s\t| {1} days'.format(mse, mse/86400))
    print('Mean Absolute Error: {0}s\t| {1} days'.format(mae, mae/86400))
    print('Median Absolute Error: {0}s\t| {1} days'.format(median, median/86400))

In [23]:
# DL distance
#distance.nlevenshtein(gt_a_label, pred_a_label) #0.1731066460587326

#damerau_levenshtein_distance(gt_a_label, pred_a_label) #784

#distance.jaccard(gt_a_label, pred_a_label) #0.4444444444444444

# Evaluation

In [24]:
max_suffix = test_len-2

In [25]:
for i in range(2, max_suffix+1):
    print('----------Suffix {}------------'.format(i))
    sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 = getSuffix(i, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)
    X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)
    pred = model.predict(X_test, verbose=0)
    pred_a = pred[0]
    pred_t = pred[1]
    print('Evaluation of Activity')
    evalAct(next_chars_2, pred[0], targetchartoindice)
    print('Evaluation of Time')
    evalTime(next_chars_t_2, pred_t, divisor)
    print('\n')
    #Add gt and pred of each suffix
    if i == 2:
        full_gt_a = next_chars_2
        full_pred_a = pred_a
        full_gt_t = next_chars_t_2
        full_pred_t = pred_t
    else:
        full_gt_a += next_chars_2
        full_pred_a = np.concatenate((full_pred_a, pred_a), axis=0)
        full_gt_t += next_chars_t_2
        full_pred_t = np.concatenate((full_pred_t, pred_t), axis=0)

----------Suffix 2------------
Evaluation of Activity
Log-loss: 0.6988266544913252
Accuracy: 76.66666666666667%
Top 3 accuracy: 98.16666666666667%
Evaluation of Time
Mean Squared Error: 367143740170.8029s	| 4249348.844569478 days
Mean Absolute Error: 293547.7551009623s	| 3.397543461816693 days
Median Absolute Error: 32906.501953125s	| 0.38086229112413195 days


----------Suffix 3------------
Evaluation of Activity
Log-loss: 1.218517665816157
Accuracy: 60.89887640449439%
Top 3 accuracy: 98.42696629213484%
Evaluation of Time
Mean Squared Error: 520221164534.5425s	| 6021078.293223871 days
Mean Absolute Error: 418958.4245569808s	| 4.84905583977987 days
Median Absolute Error: 228300.65625s	| 2.6423687065972223 days


----------Suffix 4------------
Evaluation of Activity
Log-loss: 0.9806664033195911
Accuracy: 64.1025641025641%
Top 3 accuracy: 99.48717948717949%
Evaluation of Time
Mean Squared Error: 260188138417.4542s	| 3011436.7872390533 days
Mean Absolute Error: 225169.5855068109s	| 2.6061

In [26]:
evalAct(full_gt_a, full_pred_a, targetchartoindice)

Log-loss: 0.8528703016701411
Accuracy: 71.65077772202709%
Top 3 accuracy: 98.39438033115906%


In [27]:
evalTime(full_gt_t, full_pred_t, divisor)

Mean Squared Error: 391817327106.08093s	| 4534922.767431492 days
Mean Absolute Error: 313407.9377557589s	| 3.6274066869879498 days
Median Absolute Error: 37892.171875s	| 0.4385668041087963 days


In [20]:
len(full_gt_a)

1993

# Test single case

## Predict

In [ ]:
sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 =  getSuffix(10, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)

In [ ]:
len(sentences_2), len(next_chars_2), len(sentences)

In [ ]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [ ]:
pred = model.predict(X_test, verbose=0)
pred_a1 = pred[0]
pred_t = pred[1]

In [ ]:
pred[0].shape, pred[1].shape

## Evaluate

### Activity

In [ ]:
evalAct(next_chars_2, pred[0], targetchartoindice)

### Time

In [ ]:
evalTime(next_chars_t_2, pred_t, divisor)